In [388]:
import pandas as pd
import numpy as np
from network import Network

In [389]:
recent_matches = pd.read_csv('recent_seasons_unnormalized.csv')
pca_matches = pd.read_csv('recent_seasons_PCA_99_pct_44_components.csv')
labels = pd.read_csv('labels_recent_seasons.csv')

recent_matches = recent_matches.replace(-1, np.nan)
labels = labels.replace(-1, np.nan)

In [390]:
home_odds_labels = ['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'SJH', 'VCH', 'GBH', 'BSH']
draw_odds_labels = ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'SJD', 'VCD', 'GBD', 'BSD']
away_odds_labels = ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'SJA', 'VCA', 'GBA', 'BSA']
all_odds_labels = [home_odds_labels, draw_odds_labels, away_odds_labels]
combined_odds_labels = home_odds_labels + draw_odds_labels + away_odds_labels

matches = recent_matches[combined_odds_labels]

In [391]:
# predictions = pd.DataFrame(columns=['home_prob', 'draw_prob', 'away_prob'])

# for i in range(len(matches)):
#     vals = np.array([np.random.rand() for i in range(3)])
#     vals /= np.sum(vals)
#     predictions.loc[i, 'home_prob'] = vals[0]
#     predictions.loc[i, 'draw_prob'] = vals[1]
#     predictions.loc[i, 'away_prob'] = vals[2]

In [392]:
BetNet = Network(pca_matches.shape[1])
BetNet.load_weights("weights-improvement-100-0.52.hdf5")  # Most recent weights

predictions = pd.DataFrame(columns=['home_prob', 'draw_prob', 'away_prob'])

for r, match in pca_matches.iterrows():
    net_pred = BetNet.model.predict(np.array([match.values]))[0]
    predictions.loc[r, 'home_prob'] = net_pred[0]
    predictions.loc[r, 'draw_prob'] = net_pred[1]
    predictions.loc[r, 'away_prob'] = net_pred[2]

BUILDING NETWORK...
Data Dimension:  44
ADDING REGULARIZATION...


/Users/canrobins13/Desktop/FIFABets/network.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, kernel_regularizer=<keras.reg...)`
  Dense(64, W_regularizer=regularizers.l2(0.01)))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_296 (Dense)            (None, 512)               23040     
_________________________________________________________________
activation_296 (Activation)  (None, 512)               0         
_________________________________________________________________
dense_297 (Dense)            (None, 128)               65664     
_________________________________________________________________
activation_297 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_298 (Dense)            (None, 128)               16512     
_________________________________________________________________
activation_298 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_299 (Dense)            (None, 64)                8256      
__________

In [393]:
home_, draw_, away_ = (0, 1, 2)

def get_result_corresponding_to_betting_org(org):
    for result, odds_orgs in enumerate(all_odds_labels):
        if org in odds_orgs:
            return result


def get_bet_org_highest_exp(odds, predictions):
    best_exp, best_org, best_odds, chosen_result = 0, None, 0, None
    for org, odds_ in odds.iteritems():
        result = get_result_corresponding_to_betting_org(org)
        exp = odds_ * predictions[result]
        if exp > best_exp:
            best_exp, best_org, best_odds, chosen_result = exp, org, odds_, result
            
    return chosen_result, best_org


def get_bet_org_highest_prob(odds, predictions):
    chosen_result = int(np.argmax(predictions.values))
    best_org = pd.Series.argmax(odds[all_odds_labels[chosen_result]])
    # print(best_org, ['home', 'draw', 'away'][chosen_result], odds[best_org])
    return chosen_result, best_org
    
    
def get_bet(odds, predictions, funds):
    bet_side, bet_org = get_bet_org_highest_prob(odds, predictions) # get_bet_org_highest_exp(odds, predictions)
    # bet_amount = np.sqrt(funds) * odds[bet_org] * predictions[bet_side]
    bet_amount = funds / 10
                      
    return bet_amount, bet_side, bet_org


def get_profit(bet_amount, bet_side, odds, labels_row):
    if labels_row[bet_side]:
        return bet_amount * odds - bet_amount
    else:
        return -bet_amount

In [394]:
funds = 50
initial_funds = funds

n_correct = 0

for r in range(len(matches)):
    bet_amount, bet_side, bet_org = get_bet(matches.loc[r], predictions.loc[r], funds)
    odds = matches.loc[r, bet_org]
    profit = get_profit(bet_amount, bet_side, odds, labels.loc[r])
#     print(funds, bet_amount, profit)
    funds += profit

    if funds < 1:
        break

    if profit > 0:
        n_correct += 1    
    
profit = funds - initial_funds
print('initial funds: ${:.2f}'.format(initial_funds))
print('final funds: ${:.2f}'.format(funds))
print('profit: ${0:.2f}'.format(profit))
print('profit percent: {0:.2f}%'.format(profit * 100 / initial_funds))
print('predicted {} of {} matches correctly'.format(n_correct, r))
print('prediction percent: {0:.2f}%'.format(n_correct * 100 / r))
if funds < 1:
    print('lasted {} of {} matches before dropping below $1'.format(r, len(matches)))
    print('made it through {0:.2f}% of matches before dropping below 1 dollar'.format(100 * r / len(matches)))

initial funds: $50.00
final funds: $112.21
profit: $62.21
profit percent: 124.42%
predicted 284 of 519 matches correctly
prediction percent: 54.72%
